In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from utils import *

# First Dataset: 'players.csv'

Have a look to the Datasets, one at a time.

In [ ]:
players = pd.read_csv('../dataset/players.csv')
players.info()
players

# Second Dataset: "games_details.csv"

In [ ]:
games_details = pd.read_csv('../dataset/games_details.csv', low_memory=False)
games_details.head()

# Third Dataset: "teams.csv"

In [ ]:
teams = pd.read_csv('../dataset/teams.csv')
teams.info()
teams.head()

Looking the Dataset we can notice that some arena capacity values are missing. We've decided to fill it searching the values on google:\
-Smoothie King Center: 17,805 seats;\
-Barclays Center: 17.732 seats;\
-Wells Fargo Center: 20,318 seats;\
-Talking Stick Resort Arena: 17,071 seats;

Moreover, Amway Center capacity seems to be wrong because its value is 0. So, we correct it:\
-Amway Center: 18,846 seats

In [ ]:
teams.loc[2, 'ARENACAPACITY'] = 17805.0
teams.loc[12, 'ARENACAPACITY'] = 17732.0
teams.loc[14, 'ARENACAPACITY'] = 18846.0
teams.loc[16, 'ARENACAPACITY'] = 20318.0
teams.loc[17, 'ARENACAPACITY'] = 17071.0

In [ ]:
teams = teams.drop(columns=['LEAGUE_ID', 'MAX_YEAR', 'MIN_YEAR','ABBREVIATION','NICKNAME','YEARFOUNDED','CITY','ARENA','ARENACAPACITY','OWNER','GENERALMANAGER','HEADCOACH','DLEAGUEAFFILIATION'])

# Fourth Dataset: "games.csv"

In [ ]:
games = pd.read_csv('../dataset/games.csv')
games.head()

# Fifth Dataset: "ranking.csv"
Aggiungere solo ultima partita della squadra avversaria e di quella del giocatore

In [ ]:
ranking = pd.read_csv('../dataset/ranking.csv')
ranking = ranking.drop(columns=['LEAGUE_ID', 'RETURNTOPLAY']) # se soli zeri/NaN: rimosse

# Some dataset cleaning

In [ ]:
# Start by considering games_details

# ratio_missing_values prints the percentage of missing values in the column
key1 = 'COMMENT'
ratio_missing_values_column(games_details, key1)

key2 = 'START_POSITION'
ratio_missing_values_column(games_details, key2)

'COMMENT': Since the number of valid values is very little wrt the size of the Dataset and there are not clear solutions to fill empty cells, we drop it.

'START_POSITION': same thing.

Moreover, we drop all the columns we think could lead to information leakage and also all columns that we think are not useful in the training model, such as the Nickname of the player. Our cleaned Dataset is the following

In [ ]:
Weird_rows = games_details['GAME_ID'] == 10500109
games_details = games_details[~Weird_rows]

columns_to_drop = ['MIN','COMMENT', 'PLAYER_NAME', 'NICKNAME', 'START_POSITION', 'COMMENT', 'TEAM_CITY', 'TEAM_ABBREVIATION', 'FGA', 'FG_PCT', 'FGM', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS']
games_details = games_details.drop(columns = columns_to_drop) # se non conta, si toglie. Altrimenti
# modificarla cambia il risultato
games_details = games_details.dropna()
games_details = games_details.reset_index(drop=True)

games_details.info()
games_details.head()

In [ ]:
# Perform some operations on games which will be used to add some columns to games_details
for j in range(len(games['GAME_DATE_EST'])): 
    games.loc[j, 'GAME_DATE_EST'] = StringToDate(games.loc[j, 'GAME_DATE_EST'])

games = games.sort_values(by='GAME_DATE_EST',ascending=False)
games = games.reset_index(drop=True)

In [ ]:
data = np.empty((30,6,1900)) #0 game_ids, #1 game dates, #2 won/loss, #3 counter, #4 difference between dates, #5 winrates
data[:] = np.NaN

data[:,2,0], data[:,3,0] = 0, 0

for j in range(len(games)):
    infoGame = games.loc[j,['GAME_ID','GAME_DATE_EST','HOME_TEAM_ID','VISITOR_TEAM_ID','HOME_TEAM_WINS']].tolist()
    teamH = teams['TEAM_ID'].index[teams['TEAM_ID'].eq(infoGame[2])].tolist()[0]
    teamV = teams['TEAM_ID'].index[teams['TEAM_ID'].eq(infoGame[3])].tolist()[0]
    data[teamH,0,int(data[teamH,3,0])], data[teamH,1,int(data[teamH,3,0])], data[teamH,2,int(data[teamH,3,0])] = infoGame[0], infoGame[1], infoGame[4]
    data[teamV,0,int(data[teamV,3,0])], data[teamV,1,int(data[teamV,3,0])], data[teamV,2,int(data[teamV,3,0])] = infoGame[0], infoGame[1], (1 - infoGame[4])
    data[teamH,3,0] += 1
    data[teamV,3,0] += 1

for k in range(1897):
    data[:,4,k] = data[:,1,k] - data[:,1,k+1]
    data[:,5,k] = (data[:,2,k+1] + data[:,2,k+2] + data[:,2,k+3])/3

In [ ]:
# Add columns to games_details
games_details['DATE'] = ''
games_details['DATE_DIFF'] = ''
games_details['OPPOSING_TEAM'] = ''
games_details['WINRATE'] = ''

num = 0
for j in games_details['GAME_ID']:
    gameIndex = games['GAME_ID'].index[games['GAME_ID'].eq(j)].tolist()[0]
    team = games_details['TEAM_ID'][num]
    teamIndex = teams['TEAM_ID'].index[teams['TEAM_ID'].eq(team)].tolist()[0]
    if team == games.loc[gameIndex,'HOME_TEAM_ID']:
        games_details.loc[num,'OPPOSING_TEAM'] = games.loc[gameIndex,'VISITOR_TEAM_ID']
    else:
        games_details.loc[num,'OPPOSING_TEAM'] = games.loc[gameIndex,'HOME_TEAM_ID']
    games_details.loc[num,'DATE'] = games.loc[gameIndex,'GAME_DATE_EST']
    dateIndex = np.where(data[teamIndex,0,:] == j)[0][0]
    games_details.loc[num,'DATE_DIFF'] = data[teamIndex,4,dateIndex]
    games_details.loc[num,'WINRATE'] = data[teamIndex,5,dateIndex]
    num += 1

In [ ]:
games_details = games_details.sort_values(by='DATE').reset_index(drop=True)
games_details.loc[len(games_details),'FG3M'] = pd.NA
games_details['AVERAGE_TRIPLES'] = ''

In [ ]:
lastGame = np.empty((2,len(players)))
lastGame[0,:] = 558915
lastGame[1,:] = 0

for k in range(len(games_details)-1):
    indicesList = players['PLAYER_ID'].index[players['PLAYER_ID'].eq(games_details.loc[k,'PLAYER_ID'])].tolist()
    if not indicesList:
        games_details.loc[k,'AVERAGE_TRIPLES'] = np.NaN
    else:
        playerIndex = indicesList[0]
        if lastGame[1,playerIndex] <= 1:
            games_details.loc[k,'AVERAGE_TRIPLES'] = games_details.loc[lastGame[0,playerIndex],'FG3M']
        else:
            games_details.loc[k,'AVERAGE_TRIPLES'] = (games_details.loc[lastGame[0,playerIndex],'FG3M'] + (lastGame[1,playerIndex]-1)*(games_details.loc[lastGame[0,playerIndex],'AVERAGE_TRIPLES']))/lastGame[1,playerIndex]
        lastGame[0,playerIndex] = k
        lastGame[1,playerIndex] += 1

In [ ]:
print(games[games['GAME_STATUS_TEXT']=='Final'].equals(games)) # so no useful information, Final is the content of each cell
games = games.drop(columns=['GAME_STATUS_TEXT'])

games_details = complete_games_details(games_details, games)

games_details['TEAM_ID'].astype(str)
games_details['OPPOSING_TEAM'].astype(str) # trasformare in stringhe nome così  da non lavorare
# con numeri enormi vicini ma usare l'encoder e pace

ratio_missing_values_df(games_details) # print percentage of rows in which there is at least one
# missing value
games_details.dropna(inplace=True) # drop all the rows with nans
games_details = games_details.reset_index(drop=True) # adjust the indexing

games_details.head()

In [ ]:
games_details.to_csv('../dataset/complete_dataset.csv', index=False)